In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from skimage.io import imread, imshow
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

import timm
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

In [6]:
os.getcwd()

'C:\\Users\\91760\\DRDO'

In [13]:
dir='C:\\Users\\91760\\DRDO\\Dataset_1125'

In [28]:
#categories=['Cloud','Rain','Sunrise','Foggy']
#categories=['HAZE','RAINY','SUNNY','SNOWY']
#categories=['Cloudy','Foggy','Sunny','Snowy','Rainy']
categories=['Cloud','Rain','Shine','Sunrise']
data=[]
model=timm.create_model('densenet121',pretrained=True,num_classes=10,global_pool='avg')
print(model.default_cfg)
#print(model.feature_info.module_name())
#print(model.feature_info.reduction())
#print(model.feature_info.channels())
#non,_=get_graph_node_names(model)
#print(non)
#features={'layer1.0.act2':'out'}
#feature_extractor=create_feature_extractor(model,return_nodes=features)
#print(feature_extractor)

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/densenet121_ra-50efcf5c.pth', 'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': (7, 7), 'crop_pct': 0.875, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'first_conv': 'features.conv0', 'classifier': 'classifier', 'architecture': 'densenet121'}


In [29]:
num_in_features=model.get_classifier().in_features
num_in_features

1024

In [30]:
def get_vectors(image_name):
    img = Image.open(image_name) 
    img = img.convert("RGB")
    image=torch.as_tensor(np.array(img,dtype=np.float32)).transpose(2,0)[None]
    feature_output=model.forward_features(image)
    #feature_output=feature_extractor(image)
    #feature_output=model(image)
    return feature_output

In [31]:
count=0
print(count)

0


In [32]:
for category in categories:
    path= os.path.join(dir, category)
    label=categories.index(category)
    
    for image in os.listdir(path):
        image_path=os.path.join(path,image)
        image_vector=get_vectors(image_path)
        #print(type(image_vector))
        image_list_vector=image_vector.tolist()
        image_numpy_vector=np.array(image_list_vector,dtype=object)
        #print(type(image_numpy_vector),image_numpy_vector.shape)
        r1=image_numpy_vector.shape[1]
        r2=image_numpy_vector.shape[2]
        r3=image_numpy_vector.shape[3]
        #print(r1,r2,r3)
        image_new=np.reshape(image_numpy_vector,(r1*r2*r3));
        image_new1=image_new.astype('float')
        #print(image_new1.shape)
        data.append([image_new1,label])
        count=count+1
        print(count)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 79379456 bytes.

In [19]:
print(len(data))

449


In [20]:
all_vit_models = timm.list_models('*vit*')
print(all_vit_models)

['convit_base', 'convit_small', 'convit_tiny', 'crossvit_9_240', 'crossvit_9_dagger_240', 'crossvit_15_240', 'crossvit_15_dagger_240', 'crossvit_15_dagger_408', 'crossvit_18_240', 'crossvit_18_dagger_240', 'crossvit_18_dagger_408', 'crossvit_base_240', 'crossvit_small_240', 'crossvit_tiny_240', 'levit_128', 'levit_128s', 'levit_192', 'levit_256', 'levit_384', 'vit_base_patch8_224', 'vit_base_patch8_224_in21k', 'vit_base_patch16_224', 'vit_base_patch16_224_in21k', 'vit_base_patch16_224_miil', 'vit_base_patch16_224_miil_in21k', 'vit_base_patch16_384', 'vit_base_patch16_sam_224', 'vit_base_patch32_224', 'vit_base_patch32_224_in21k', 'vit_base_patch32_384', 'vit_base_patch32_sam_224', 'vit_base_r26_s32_224', 'vit_base_r50_s16_224', 'vit_base_r50_s16_224_in21k', 'vit_base_r50_s16_384', 'vit_base_resnet26d_224', 'vit_base_resnet50_224_in21k', 'vit_base_resnet50_384', 'vit_base_resnet50d_224', 'vit_giant_patch14_224', 'vit_gigantic_patch14_224', 'vit_huge_patch14_224', 'vit_huge_patch14_224_i

In [22]:
all_densenet_models = timm.list_models('*densenet*')
print(all_densenet_models)

['densenet121', 'densenet121d', 'densenet161', 'densenet169', 'densenet201', 'densenet264', 'densenet264d_iabn', 'densenetblur121d', 'tv_densenet121']
